# 🔊 AI Generated Ad Films for Product Videography | VideoDB
<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Voiceover_Ad_Films.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

Creating professional product advertisements typically requires a team: copywriters for the script, voice actors for the narration, and editors to stitch it all together.

**VideoDB** streamlines this into a single, automated workflow.

In this tutorial, we will:
1.  **Upload** raw product footage (a jewelry shoot).
2.  **Analyze** the visual content automatically.
3.  **Generate** a professional ad script based on the visuals.
4.  **Synthesize** a high-quality voiceover.
5.  **Publish** the final commercial.


## Setup

---
### 📦  Installing packages

In [ ]:
%pip -q install videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### 🔑 API Key
You only need your **VideoDB API Key**.
> Get your API key from [VideoDB Console](https://console.videodb.io). (Free for first 50 uploads, **No credit card required**).

In [ ]:
import videodb
import os
from getpass import getpass

# Prompt user for API key securely
api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

Please enter your VideoDB API Key: ··········


## Implementation

---
### 🌐 Step 1: Connect to VideoDB
Establish a connection to your VideoDB project.

In [ ]:
from videodb import connect

# Connect to VideoDB
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Product Footage
We'll upload a raw video clip of a jewelry product shoot from YouTube.

In [ ]:
# Upload a video by URL
video = coll.upload(url='https://www.youtube.com/watch?v=2DcAMbmmYNM')
print(f"Uploaded Video ID: {video.id}")

Uploaded Video ID: m-z-019bf664-b9c6-7ae0-86e8-ab7e594cee51


### 🔍 Step 3: Analyze Visuals
To write a relevant script, we first need to understand what's in the video. We'll use `index_scenes()` to extract detailed descriptions of the visual content.

In [ ]:
print("Indexing scenes... this might take a moment.")
scene_id = video.index_scenes()
print(scene_id)

Indexing scenes... this might take a moment.
4ec75752583d27ea


Let's view the description of first scene of the video

In [ ]:
import json
scenes = video.get_scene_index(scene_id)

print(json.dumps(scenes[0], indent=2))

{
  "description": "The entire series of images presents a uniform and absolute darkness, an unbroken expanse of pure black. There are no discernible shapes, colors, or details to be found, suggesting a complete absence of light or any visual information whatsoever across the whole sequence.",
  "end": 1.401,
  "metadata": {},
  "scene_metadata": {},
  "start": 0.0
}


### 📝 Step 4: Generate Ad Script
Now we use VideoDB's text generation to write the commercial. We feed the visual descriptions into the prompt to ensure the script perfectly matches the mood of the footage.

In [ ]:
# Construct a prompt with the scene context
scene_context = "\n".join([f"- {scene['description']}" for scene in scenes])

prompt = f"""
Here is a visual description of a jewelry product video:
{scene_context}

Write a short, elegant, and luxurious voiceover script for this video advertisement.
- Tone: Sophisticated, calming, premium.
- Length: Short (approx 20 seconds of speech).
- Content: Focus on beauty, craftsmanship, and elegance.
- Format: Return ONLY the raw narration text.
"""

# Generate script
text_response = coll.generate_text(
    prompt=prompt,
    model_name="pro"
)

ad_script = text_response["output"]

print("--- Generated Ad Script ---")
print(ad_script)

--- Generated Ad Script ---
In every stone, a story. In every design, a soul. We capture the fire of the earth and the light of the stars, crafting not just jewelry, but a reflection of your own brilliance. Wear your legacy.


### 🎙️ Step 5: Generate Voiceover
We'll turn the script into audio.

In [ ]:
print("Generating voiceover audio...")

# Generate speech directly as a VideoDB Audio Asset
audio = coll.generate_voice(
    text=ad_script,
    voice_name="Default"
)

print(f"Generated Audio Asset ID: {audio.id}")

Generating voiceover audio...
Generated Audio Asset ID: a-z-019bf66d-9976-72c2-b453-ec58bf30d032


### 🎬 Step 6: Compose the Advertisement
We'll overlay the generated voiceover onto the original video using the Timeline editor.

In [ ]:
from videodb.editor import Timeline, Track, Clip, VideoAsset, AudioAsset

# Create a timeline
timeline = Timeline(conn)

# 1. Video Track (Background)
video_track = Track()
video_asset = VideoAsset(id=video.id)
video_clip = Clip(asset=video_asset, duration=float(video.length))
video_track.add_clip(0, video_clip)
timeline.add_track(video_track)

# 2. Audio Track (Voiceover Overlay)
audio_track = Track()
audio_asset = AudioAsset(id=audio.id)
audio_clip = Clip(asset=audio_asset, duration=float(audio.length))
audio_track.add_clip(0, audio_clip)
timeline.add_track(audio_track)

### 🪄 Step 7: Review and Share
Generate the stream URL to watch your AI-created commercial.

In [ ]:
from videodb import play_stream

stream_url = timeline.generate_stream()
print(f"Stream URL: {stream_url}")
play_stream(stream_url)

Stream URL: https://play.videodb.io/v1/e5f488c4-f6ad-4b16-a462-9c3f46af5d86.m3u8


---

### 🎉 Conclusion

You have successfully automated the production of a product advertisement.

By replacing multiple external tools with VideoDB's unified SDK, you can now build scalable video generation engines that turn raw footage into polished content automatically.

Explore more at [docs.videodb.io](https://docs.videodb.io/).